# **FORECAST CREDITOS GENERALES**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "vallarta" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_vallarta',
 'df_financiamientos_2020_vallarta',
 'df_financiamientos_2021_vallarta',
 'df_financiamientos_2022_vallarta',
 'df_financiamientos_2023_vallarta',
 'df_financiamientos_2024_vallarta']

# **Forecast créditos generales (cantidad)**

### BASE

In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [3]:
# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y contar el número de registros únicos
    total_creditos_df = df.groupby(['mes']).size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'num_registros']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_conteo_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_conteo_creditos_pachuca

,año,mes,num_registros
0,2019,1,132
1,2019,2,145
2,2019,3,182
3,2019,4,146
4,2019,5,194
...,...,...,...
66,2024,7,109
67,2024,8,108
68,2024,9,89
69,2024,10,114


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Preparar datos para el modelo
X = total_conteo_creditos_pachuca[['año', 'mes']]
y = total_conteo_creditos_pachuca['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [5]:
from sklearn.metrics import r2_score
import numpy as np

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)
# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación (R^2):", r2)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print("Error absoluto medio (MAE):", mae)
rmse = np.sqrt(mse)
print("Raíz del error cuadrático medio (RMSE):", rmse)

Coeficiente de determinación (R^2): 0.5975329801207998
Error cuadrático medio (MSE): 110.88582418620759
Error absoluto medio (MAE): 9.118891262751017
Raíz del error cuadrático medio (RMSE): 10.53023381441303


In [6]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)

# Redondear las predicciones a números enteros
predicciones_enteros = np.round(predicciones).astype(int)

# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros
nuevas_fechas.head()

total_conteo_pachuca = pd.concat([total_conteo_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_conteo_pachuca

,año,mes,num_registros
0,2019,1,78
1,2019,2,87
2,2019,3,88
3,2019,4,79
4,2019,5,103
...,...,...,...
92,2026,8,138
93,2026,9,141
94,2026,10,145
95,2026,11,148


In [7]:
# Asegurarse de que la columna 'año' es de tipo entero
#total_conteo_pachuca['año'] = total_conteo_pachuca['año'].astype(int)

total_conteo_pachuca = total_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_conteo_pachuca['modalidad'] = 'Créditos (todos)'
#total_conteo_pachuca = total_conteo_pachuca.groupby('año').agg({'num_registros': 'sum', 'modalidad': 'first'}).reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales
# Formatear la columna 'montos' con la función personalizada
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].map(format_number)
total_conteo_pachuca = total_conteo_pachuca.iloc[1:].reset_index(drop=True)

total_conteo_pachuca

,año,num_registros,modalidad
0,2025,"1,543",Créditos (todos)
1,2026,"1,599",Créditos (todos)
2,2019,"1,215",Créditos (todos)
3,2020,"1,178",Créditos (todos)
4,2021,"1,430",Créditos (todos)
5,2022,"1,388",Créditos (todos)
6,2023,"1,399",Créditos (todos)
7,2024,792,Créditos (todos)


In [8]:
# Valores conocidos
registros_2022 = 1388
registros_2023 = 1399

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))


Estimación de registros en 2024: 1410


In [9]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_conteo_pachuca['num_registros'] = total_conteo_pachuca['num_registros'].replace({',': ''}, regex=True).astype(int)

# Ahora intenta realizar la sustitución
total_conteo_pachuca.loc[total_conteo_pachuca['año'] == "2024", 'num_registros'] = 1410

# Verifica el cambio en el DataFrame
print(total_conteo_pachuca)


    año  num_registros         modalidad
0  2025           1543  Créditos (todos)
1  2026           1599  Créditos (todos)
2  2019           1215  Créditos (todos)
3  2020           1178  Créditos (todos)
4  2021           1430  Créditos (todos)
5  2022           1388  Créditos (todos)
6  2023           1399  Créditos (todos)
7  2024           1410  Créditos (todos)


### Modalidad 1: Nueva

In [10]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y contar el número de registros únicos por modalidad en cada año
    conteo_creditos_año = df.groupby('modalidad').size().reset_index(name='num_registros')
    # Agregar una columna con el año correspondiente
    conteo_creditos_año['año'] = año
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,num_registros,año
0,1,620,2019
1,2,75,2019
2,3,454,2019
3,4,66,2019
4,1,629,2020
5,2,94,2020
6,3,385,2020
7,4,70,2020
8,1,699,2021
9,2,89,2021


In [11]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y contar el número de registros
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año']).size().reset_index(name='num_registros')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='num_registros', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes   1   2   3   4   5   6   7   8   9   10  11  12
 año                                                 
 2019  50  49  50  38  44  53  50  58  55  48  63  62
 2020  45  44  58  26  43  52  57  54  79  48  53  70
 2021  49  61  75  53  53  55  70  53  56  55  49  70
 2022  57  39  55  51  43  45  53  48  58  42  65  68
 2023  40  24  51  38  61  63  42  48  39  55  47  70
 2024  34  33  41  36  29  55  45   0   0   0   0   0,
 mes   1   2   3   4   5   6   7   8   9   10  11  12
 año                                                 
 2019  19  30  30  32  51  39  36  34  36  52  39  56
 2020  31  30  28  12  24  30  31  30  35  45  45  44
 2021  23  35  46  57  45  59  60  42  43  33  42  50
 2022  26  34  54  46  39  42  52  31  48  41  34  70
 2023  28  35  43  32  47  50  38  33  43  43  38  51
 2024  28  38  40  40  41  42  60   0   0   0   0   0)

In [12]:
X = df_modalidad_1[['año', 'mes']]
y = df_modalidad_1['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [13]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024,6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 


# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

nueva_conteo_pachuca = pd.concat([df_modalidad_1, nuevas_fechas], ignore_index=True)
nueva_conteo_pachuca = nueva_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
nueva_conteo_pachuca['modalidad'] = 'Nueva'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo_pachuca['num_registros'] = nueva_conteo_pachuca['num_registros'].map(format_number)
nueva_conteo_pachuca

,año,num_registros,modalidad
0,2019,620,Nueva
1,2020,629,Nueva
2,2021,699,Nueva
3,2022,624,Nueva
4,2023,578,Nueva
5,2024,587,Nueva
6,2025,554,Nueva
7,2026,535,Nueva


In [14]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['num_registros']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [15]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026]) # Concatenamos 


# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['num_registros'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['num_registros'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,num_registros,modalidad
0,2019,454,Usada
1,2020,385,Usada
2,2021,535,Usada
3,2022,517,Usada
4,2023,481,Usada
5,2024,586,Usada
6,2025,568,Usada
7,2026,591,Usada


In [16]:
conteo_pachuca = pd.concat([
    total_conteo_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [17]:
# Convertir 'num_registros' a enteros y eliminar comas
conteo_pachuca['num_registros'] = conteo_pachuca['num_registros'].apply(lambda x: int(str(x).replace(',', '')))

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año  num_registros         modalidad
2  2019           1215  Créditos (todos)
0  2019            620             Nueva
0  2019            454             Usada
1  2020            629             Nueva
1  2020            385             Usada
3  2020           1178  Créditos (todos)
4  2021           1430  Créditos (todos)
2  2021            699             Nueva
2  2021            535             Usada
3  2022            517             Usada
3  2022            624             Nueva
5  2022           1388  Créditos (todos)
4  2023            578             Nueva
6  2023           1399  Créditos (todos)
4  2023            481             Usada
7  2024           1410  Créditos (todos)
5  2024            587             Nueva
5  2024            586             Usada
0  2025           1543  Créditos (todos)
6  2025            554             Nueva
6  2025            568             Usada
1  2026           1599  Créditos (todos)
7  2026            535             Nueva
7  2026         

In [18]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['num_registros'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['num_registros'],  # Añadir el texto de los valores
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    #yaxis_title="Número de Registros",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Configurar el eje Y para mostrar los valores completos en miles (sin 'K')
fig.update_yaxes(tickformat=",")  # ',' para formato de miles sin abreviación


# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_cantidad', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [TOTAL]**

In [19]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).sum().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_suma_creditos_pachuca

,año,mes,montos
0,2019,1,1.027393e+08
1,2019,2,9.969129e+07
2,2019,3,1.120002e+08
3,2019,4,9.428530e+07
4,2019,5,1.177026e+08
...,...,...,...
62,2024,3,1.488332e+08
63,2024,4,1.764134e+08
64,2024,5,1.322260e+08
65,2024,6,1.996509e+08


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos_pachuca[['año', 'mes']]
y = total_suma_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [21]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
 #                                 'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)

total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

,año,montos,modalidad
0,2025,"2,460,935,621.62",Créditos (todos)
1,2026,"2,613,593,246.12",Créditos (todos)
2,2019,"1,476,067,496.32",Créditos (todos)
3,2020,"1,596,425,866.54",Créditos (todos)
4,2021,"2,050,344,728.76",Créditos (todos)
5,2022,"2,184,952,825.06",Créditos (todos)
6,2023,"2,004,131,940.26",Créditos (todos)
7,2024,"1,142,612,273.19",Créditos (todos)


In [22]:
# Valores conocidos
registros_2022 = 2184952825.06
registros_2023 = 2004131940.26

# Calcula la tasa de cambio porcentual de 2022 a 2023
tasa_cambio = (registros_2023 - registros_2022) / registros_2022

# Aplica la misma tasa de cambio para estimar 2024 completo
estimacion_2024 = registros_2023 * (1 + tasa_cambio)

print("Estimación de registros en 2024:", round(estimacion_2024))

Estimación de registros en 2024: 1838275311


In [35]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 1838275311


# Verifica el cambio en el DataFrame
print(total_todos_pachuca)


    año      montos         modalidad
0  2025  2460935621  Créditos (todos)
1  2026  2613593246  Créditos (todos)
2  2019  1476067496  Créditos (todos)
3  2020  1596425866  Créditos (todos)
4  2021  2050344728  Créditos (todos)
5  2022  2184952825  Créditos (todos)
6  2023  2004131940  Créditos (todos)
7  2024  1838275311  Créditos (todos)


In [36]:
df_financiamientos_2024_queretaro.shape

(5435, 17)

In [23]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].sum().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, suma_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,montos,año
0,1,1.030751e+09,2019
1,2,5.098355e+06,2019
2,3,3.792651e+08,2019
3,4,6.095277e+07,2019
4,1,1.079870e+09,2020
5,2,1.814705e+07,2020
6,3,4.172683e+08,2020
7,4,8.114009e+07,2020
8,1,1.224540e+09,2021
9,2,2.209514e+07,2021


In [26]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].sum().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes            1             2             3            4             5   \
 año                                                                        
 2019  79550172.37  6.865934e+07  7.813391e+07  65004112.92  7.042122e+07   
 2020  89631712.73  7.330905e+07  8.646081e+07  46861696.45  6.383653e+07   
 2021  86093989.82  1.021271e+08  1.310891e+08  82826969.29  1.089669e+08   
 2022  87645135.00  6.452677e+07  8.516246e+07  89683591.79  9.198135e+07   
 2023  81253823.00  4.348250e+07  8.057166e+07  84896216.96  1.026137e+08   
 2024  63426443.91  7.851500e+07  7.576550e+07  84832277.65  6.007310e+07   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  8.746573e+07  8.074711e+07  1.045717e+08  9.784357e+07  7.894973e+07   
 2020  8.994808e+07  8.193729e+07  9.771782e+07  1.195538e+08  8.715167e+07   
 2021  7.237658e+07  1.311713e+08  7.860909e+07  1.082944e+08  8.8

In [27]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]

In [28]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo_pachuca = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].sum().reset_index()

# Mostrar el DataFrame resultante
print(nueva_conteo_pachuca)

    año modalidad        montos
0  2019     Nueva  1.030751e+09
1  2020     Nueva  1.079870e+09
2  2021     Nueva  1.224540e+09
3  2022     Nueva  1.207404e+09
4  2023     Nueva  1.168614e+09
5  2024     Nueva  1.297059e+09
6  2025     Nueva  1.256650e+09
7  2026     Nueva  1.286929e+09


In [29]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [30]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['montos'].sum().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
#def format_number(x):
 #   return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
#mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,montos,modalidad
0,2019,3.792651e+08,Usada
1,2020,4.172683e+08,Usada
2,2021,6.282082e+08,Usada
3,2022,8.383946e+08,Usada
4,2023,6.739793e+08,Usada
5,2024,9.235158e+08,Usada
6,2025,9.733274e+08,Usada
7,2026,1.070689e+09,Usada


In [36]:
conteo_pachuca = pd.concat([
    total_todos_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [37]:
conteo_pachuca

,año,montos,modalidad
0,2025,2.460936e+09,Créditos (todos)
1,2026,2.613593e+09,Créditos (todos)
2,2019,1.476067e+09,Créditos (todos)
3,2020,1.596426e+09,Créditos (todos)
4,2021,2.050345e+09,Créditos (todos)
5,2022,2.184953e+09,Créditos (todos)
6,2023,2.004132e+09,Créditos (todos)
7,2024,1.838275e+09,Créditos (todos)
0,2019,1.030751e+09,Nueva
1,2020,1.079870e+09,Nueva


In [38]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año      montos         modalidad
2  2019  1476067496  Créditos (todos)
0  2019  1030751267             Nueva
0  2019   379265106             Usada
1  2020  1079870382             Nueva
1  2020   417268347             Usada
3  2020  1596425866  Créditos (todos)
4  2021  2050344728  Créditos (todos)
2  2021  1224540479             Nueva
2  2021   628208169             Usada
3  2022   838394556             Usada
3  2022  1207403961             Nueva
5  2022  2184952825  Créditos (todos)
4  2023  1168614346             Nueva
6  2023  2004131940  Créditos (todos)
4  2023   673979349             Usada
7  2024  1838275311  Créditos (todos)
5  2024  1297058914             Nueva
5  2024   923515826             Usada
0  2025  2460935621  Créditos (todos)
6  2025  1256650394             Nueva
6  2025   973327350             Usada
1  2026  2613593246  Créditos (todos)
7  2026  1286928599             Nueva
7  2026  1070688699             Usada


In [39]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [1_000, 1_000_000, 1_000_000_000,5_000_000_000,10_000_000_000,15_000_000_000,20_000_000_000,25_000_000_000, 30_000_000_000]  # Valores hasta 30 mil millones
ticktext = [formatear_numero(val) for val in tickvals]  # Formatea cada valor en palabras

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()


# **3.4 Forecast Monto de créditos generales. [PROMEDIO]**

In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).mean().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos = pd.concat(dfs, ignore_index=True)
total_suma_creditos

,año,mes,montos
0,2019,1,1.317171e+06
1,2019,2,1.145877e+06
2,2019,3,1.272730e+06
3,2019,4,1.193485e+06
4,2019,5,1.142744e+06
...,...,...,...
62,2024,3,1.250699e+06
63,2024,4,1.633457e+06
64,2024,5,1.259296e+06
65,2024,6,1.623178e+06


In [5]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, suma_creditos_año], ignore_index=True)

total_conteo_creditos

,modalidad,montos,año
0,1,1.662502e+06,2019
1,2,6.797807e+04,2019
2,3,8.353857e+05,2019
3,4,9.235268e+05,2019
4,1,1.716805e+06,2020
5,2,1.930537e+05,2020
6,3,1.083814e+06,2020
7,4,1.159144e+06,2020
8,1,1.751846e+06,2021
9,2,2.482600e+05,2021


In [6]:
total_conteo_creditos = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos = pd.concat([total_conteo_creditos, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos[total_conteo_creditos['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  1.591003e+06  1.401211e+06  1.562678e+06  1.710635e+06  1.600482e+06   
 2020  1.991816e+06  1.666115e+06  1.490704e+06  1.802373e+06  1.484571e+06   
 2021  1.757020e+06  1.674215e+06  1.747855e+06  1.562773e+06  2.055978e+06   
 2022  1.537634e+06  1.654533e+06  1.548408e+06  1.758502e+06  2.139101e+06   
 2023  2.031346e+06  1.811771e+06  1.579837e+06  2.234111e+06  1.682192e+06   
 2024  1.865484e+06  2.379243e+06  1.847939e+06  2.356452e+06  2.071486e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  1.650297e+06  1.614942e+06  1.802961e+06  1.778974e+06  1.644786e+06   
 2020  1.729771e+06  1.437496e+06  1.809589e+06  1.513340e+06  1.815660e+06   
 2021  1.315938e+06  1.873876e+06  1.483190e+06  1

In [7]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]
modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_4476\3999403270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modalidad1['montos'] = modalidad1['montos'].replace({',': ''}, regex=True).astype(float).astype(int)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = modalidad1[['año', 'mes']]
y = modalidad1['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [13]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
nueva_conteo['montos'] = nueva_conteo['montos'].map(format_number)

# Mostrar el DataFrame resultante
print(nueva_conteo)

    año modalidad   montos
0  2019     Nueva  1656062
1  2020     Nueva  1722048
2  2021     Nueva  1742330
3  2022     Nueva  1945195
4  2023     Nueva  2043088
5  2024     Nueva  2238553
6  2025     Nueva  2301376
7  2026     Nueva  2418399


In [14]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [15]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo = mejoramiento_conteo.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
def format_number(x):
    return '{:.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
mejoramiento_conteo['montos'] = mejoramiento_conteo['montos'].map(format_number)
mejoramiento_conteo

,año,montos,modalidad
0,2019,850138,Usada
1,2020,1050446,Usada
2,2021,1163000,Usada
3,2022,1602180,Usada
4,2023,1409704,Usada
5,2024,1593760,Usada
6,2025,1783425,Usada
7,2026,1928447,Usada


In [16]:
conteo_pachuca = pd.concat([
    #total_todos,
    nueva_conteo,
    mejoramiento_conteo,
])

In [17]:
conteo_pachuca = conteo_pachuca.round(2)
conteo_pachuca

,año,modalidad,montos
0,2019,Nueva,1656062
1,2020,Nueva,1722048
2,2021,Nueva,1742330
3,2022,Nueva,1945195
4,2023,Nueva,2043088
5,2024,Nueva,2238553
6,2025,Nueva,2301376
7,2026,Nueva,2418399
0,2019,Usada,850138
1,2020,Usada,1050446


In [18]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año modalidad   montos
0  2019     Nueva  1656062
0  2019     Usada   850138
1  2020     Usada  1050446
1  2020     Nueva  1722048
2  2021     Nueva  1742330
2  2021     Usada  1163000
3  2022     Usada  1602180
3  2022     Nueva  1945195
4  2023     Usada  1409704
4  2023     Nueva  2043088
5  2024     Nueva  2238553
5  2024     Usada  1593760
6  2025     Usada  1783425
6  2025     Nueva  2301376
7  2026     Nueva  2418399
7  2026     Usada  1928447


In [24]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
modalidades = conteo_pachuca['modalidad'].unique()
#colores = ['red', 'lightgreen']  # Definir los colores a usar
for i, modalidad in enumerate(modalidades):
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center",  # Posicionar el texto encima de los puntos
        #line=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
        #marker=dict(color=colores[i % len(colores)]),  # Asignar el color basado en el índice
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto promedio anual",
    yaxis_tickformat=",.0f",  # Formatear el eje y con separadores de coma y sin decimales
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)
# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()

# **3.4 Forecast Monto de créditos generales. [TOTAL]**

In [80]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id', 'año', 'mes', 'modalidad', y 'monto'
    segment_df = globals()[nombre_df][['id','año','mes','modalidad','monto']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Lista para almacenar los DataFrames procesados
dfs = []

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'mes', y sumar los montos
    total_creditos_df = df.groupby(['mes']).sum().reset_index()
    # Renombrar la columna 'monto' a 'montos'
    total_creditos_df.rename(columns={'monto': 'montos'}, inplace=True)
    # Agregar una columna con el año correspondiente
    total_creditos_df['año'] = año
    # Seleccionar las columnas necesarias
    total_creditos_df = total_creditos_df[['año', 'mes', 'montos']]
    
    # Añadir el DataFrame procesado a la lista
    dfs.append(total_creditos_df)

# Concatenar todos los DataFrames procesados en uno solo
total_suma_creditos_pachuca = pd.concat(dfs, ignore_index=True)
total_suma_creditos_pachuca

,año,mes,montos
0,2019,1,1.027393e+08
1,2019,2,9.969129e+07
2,2019,3,1.120002e+08
3,2019,4,9.428530e+07
4,2019,5,1.177026e+08
...,...,...,...
62,2024,3,1.488332e+08
63,2024,4,1.764134e+08
64,2024,5,1.322260e+08
65,2024,6,1.996509e+08


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Preparar datos para el modelo
X = total_suma_creditos_pachuca[['año', 'mes']]
y = total_suma_creditos_pachuca['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [82]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})

# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
 #                                 'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([total_suma_creditos_pachuca, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
total_todos_pachuca['modalidad'] = 'Créditos (todos)'
# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)

total_todos_pachuca = total_todos_pachuca.iloc[1:].reset_index(drop=True)

total_todos_pachuca

,año,montos,modalidad
0,2025,"205,077,968.47",Créditos (todos)
1,2026,"217,799,437.18",Créditos (todos)
2,2019,"123,005,624.69",Créditos (todos)
3,2020,"133,035,488.88",Créditos (todos)
4,2021,"170,862,060.73",Créditos (todos)
5,2022,"182,079,402.09",Créditos (todos)
6,2023,"167,010,995.02",Créditos (todos)
7,2024,"163,230,324.74",Créditos (todos)


In [83]:
# Remover comas y convertir la columna 'num_registros' a enteros
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Ahora intenta realizar la sustitución
#total_todos_pachuca.loc[total_todos_pachuca['año'] == "2024", 'montos'] = 2257062

# Verifica el cambio en el DataFrame
print(total_todos_pachuca)

    año     montos         modalidad
0  2025  205077968  Créditos (todos)
1  2026  217799437  Créditos (todos)
2  2019  123005624  Créditos (todos)
3  2020  133035488  Créditos (todos)
4  2021  170862060  Créditos (todos)
5  2022  182079402  Créditos (todos)
6  2023  167010995  Créditos (todos)
7  2024  163230324  Créditos (todos)


In [87]:
df_financiamientos_2024_queretaro['monto'].mean().round()

np.float64(2257062.0)

In [88]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    # Agrupar por 'modalidad' y sumar el 'monto' por modalidad en cada año
    suma_creditos_año = df.groupby('modalidad')['monto'].mean().reset_index(name='montos')
    # Agregar una columna con el año correspondiente
    suma_creditos_año['año'] = año
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, suma_creditos_año], ignore_index=True)

total_conteo_creditos_pachuca

,modalidad,montos,año
0,1,1.662502e+06,2019
1,2,6.797807e+04,2019
2,3,8.353857e+05,2019
3,4,9.235268e+05,2019
4,1,1.716805e+06,2020
5,2,1.930537e+05,2020
6,3,1.083814e+06,2020
7,4,1.159144e+06,2020
8,1,1.751846e+06,2021
9,2,2.482600e+05,2021


In [89]:
total_conteo_creditos_pachuca = pd.DataFrame()

# Iterar sobre la lista de DataFrames y aplicar el procesamiento a cada uno
for i, df in enumerate(dataframes_list):
    # Extraer el año del nombre del DataFrame
    año = nombres_df_filtrados[i].split('_')[2]
    
    # Agrupar por 'modalidad', 'mes', y 'año' y sumar el 'monto' por modalidad en cada año
    conteo_creditos_año_mes = df.groupby(['modalidad', 'mes', 'año'])['monto'].mean().reset_index(name='montos')
    
    # Concatenar el DataFrame procesado al DataFrame final
    total_conteo_creditos_pachuca = pd.concat([total_conteo_creditos_pachuca, conteo_creditos_año_mes], ignore_index=True)

# Filtrar los datos por modalidad
df_modalidad_1 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 1]
df_modalidad_3 = total_conteo_creditos_pachuca[total_conteo_creditos_pachuca['modalidad'] == 3]

# Pivotar los DataFrames para que cada columna sea el número de registros por mes
df_modalidad_1_pivot = df_modalidad_1.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)
df_modalidad_3_pivot = df_modalidad_3.pivot_table(index=['año'], columns=['mes'], values='montos', aggfunc='sum', fill_value=0)

# Mostrar los DataFrames resultantes
df_modalidad_1_pivot, df_modalidad_3_pivot

(mes             1             2             3             4             5   \
 año                                                                          
 2019  1.591003e+06  1.401211e+06  1.562678e+06  1.710635e+06  1.600482e+06   
 2020  1.991816e+06  1.666115e+06  1.490704e+06  1.802373e+06  1.484571e+06   
 2021  1.757020e+06  1.674215e+06  1.747855e+06  1.562773e+06  2.055978e+06   
 2022  1.537634e+06  1.654533e+06  1.548408e+06  1.758502e+06  2.139101e+06   
 2023  2.031346e+06  1.811771e+06  1.579837e+06  2.234111e+06  1.682192e+06   
 2024  1.865484e+06  2.379243e+06  1.847939e+06  2.356452e+06  2.071486e+06   
 
 mes             6             7             8             9             10  \
 año                                                                          
 2019  1.650297e+06  1.614942e+06  1.802961e+06  1.778974e+06  1.644786e+06   
 2020  1.729771e+06  1.437496e+06  1.809589e+06  1.513340e+06  1.815660e+06   
 2021  1.315938e+06  1.873876e+06  1.483190e+06  1

In [90]:
modalidad1 = df_modalidad_1#[df_modalidad_1['año'] != 2024]

In [91]:
# Crear DataFrame de fechas para predicciones solo en 2025 y 2026
fechas_prediccion = pd.DataFrame({
    'año': np.repeat([2025, 2026], 12),
    'mes': np.tile(range(1, 13), 2)
})

# Crear DataFrame de fechas para predicciones de los faltantes de 2023
fechas_2023 = pd.DataFrame({
    'año': np.repeat(2024, 6),
    'mes': range(7, 13)
})

# Concatenar los DataFrames de fechas
fechas_prediccion = pd.concat([fechas_2023, fechas_prediccion], ignore_index=True)

# Entrenar el modelo solo con datos completos hasta 2023
X_train = modalidad1[['año', 'mes']]
y_train = modalidad1['montos']
modelo.fit(X_train, y_train)

# Realizar predicciones para 2023, 2025 y 2026
predicciones = modelo.predict(fechas_prediccion)
predicciones_enteros = np.round(predicciones).astype(int)

# Agregar las predicciones al DataFrame de fechas futuras
fechas_prediccion['montos'] = predicciones_enteros

# Concatenar los datos originales y las predicciones
df_con_predicciones = pd.concat([df_modalidad_1, fechas_prediccion], ignore_index=True)
df_con_predicciones['modalidad'] = 'Nueva'

# Agrupar para obtener la suma anual por modalidad
nueva_conteo_pachuca = df_con_predicciones.groupby(['año', 'modalidad'])['montos'].mean().reset_index()

# Mostrar el DataFrame resultante
print(nueva_conteo_pachuca)

    año modalidad        montos
0  2019     Nueva  1.656063e+06
1  2020     Nueva  1.722049e+06
2  2021     Nueva  1.742330e+06
3  2022     Nueva  1.945195e+06
4  2023     Nueva  2.043088e+06
5  2024     Nueva  2.238553e+06
6  2025     Nueva  2.301376e+06
7  2026     Nueva  2.418400e+06


In [92]:
X = df_modalidad_3[['año', 'mes']]
y = df_modalidad_3['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [93]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
#fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 5),
 #                           'mes': range(8, 13)})

fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
# DF con todas las combinaciones posibles de año y mes para 2024-2026
#fechas_2024_2026 = pd.DataFrame({'año': np.repeat(range(2024, 2027), 12),
#                                  'mes': np.tile(range(1, 13), 3)})
#nuevas_fechas = pd.concat([fechas_2023, fechas_2024_2026]) # Concatenamos 

# DF con todas las combinaciones posibles de año y mes para 2025-2026
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2024, fechas_2025_2026]) # Concatenamos 

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
predicciones_enteros = np.round(predicciones).astype(int) # Redondear las predicciones a números enteros
# Agregamos las predicciones redondeadas al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones_enteros

mejoramiento_conteo_pachuca = pd.concat([df_modalidad_3, nuevas_fechas], ignore_index=True)
mejoramiento_conteo_pachuca = mejoramiento_conteo_pachuca.groupby('año')['montos'].mean().reset_index()
# Agregar una nueva columna 'vivienda' con el valor 'Usada'
mejoramiento_conteo_pachuca['modalidad'] = 'Usada'
# Función para formatear números con separadores de coma y sin decimales
#def format_number(x):
 #   return '{:,.0f}'.format(x)  # Formatear sin decimales

# Formatear la columna 'num_registros' con la función personalizada
#mejoramiento_conteo_pachuca['num_registros'] = mejoramiento_conteo_pachuca['num_registros'].map(format_number)
mejoramiento_conteo_pachuca

,año,montos,modalidad
0,2019,8.501377e+05,Usada
1,2020,1.050446e+06,Usada
2,2021,1.163000e+06,Usada
3,2022,1.602180e+06,Usada
4,2023,1.409704e+06,Usada
5,2024,1.593760e+06,Usada
6,2025,1.783425e+06,Usada
7,2026,1.928447e+06,Usada


In [94]:
conteo_pachuca = pd.concat([
    total_todos_pachuca,
    nueva_conteo_pachuca,
    mejoramiento_conteo_pachuca,
])

In [95]:
conteo_pachuca

,año,montos,modalidad
0,2025,2.050780e+08,Créditos (todos)
1,2026,2.177994e+08,Créditos (todos)
2,2019,1.230056e+08,Créditos (todos)
3,2020,1.330355e+08,Créditos (todos)
4,2021,1.708621e+08,Créditos (todos)
5,2022,1.820794e+08,Créditos (todos)
6,2023,1.670110e+08,Créditos (todos)
7,2024,1.632303e+08,Créditos (todos)
0,2019,1.656063e+06,Nueva
1,2020,1.722049e+06,Nueva


In [96]:
# Convertir 'montos' a enteros y eliminar comas, manejo de NaN
conteo_pachuca['montos'] = conteo_pachuca['montos'].apply(lambda x: int(float(x)) if pd.notna(x) else x)

# Convertir 'año' a enteros para asegurar el orden correcto
conteo_pachuca['año'] = conteo_pachuca['año'].astype(int)

# Ordenar los años de acuerdo con el orden específico en 'orden_anos'
orden_anos = [2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]
conteo_pachuca = conteo_pachuca.sort_values(by='año', key=lambda x: x.map({year: i for i, year in enumerate(orden_anos)}))

# Ahora el DataFrame debería estar ordenado correctamente
print(conteo_pachuca)

    año     montos         modalidad
2  2019  123005624  Créditos (todos)
0  2019    1656062             Nueva
0  2019     850137             Usada
1  2020    1722048             Nueva
1  2020    1050446             Usada
3  2020  133035488  Créditos (todos)
4  2021  170862060  Créditos (todos)
2  2021    1742330             Nueva
2  2021    1162999             Usada
3  2022    1602179             Usada
3  2022    1945195             Nueva
5  2022  182079402  Créditos (todos)
4  2023    2043088             Nueva
6  2023  167010995  Créditos (todos)
4  2023    1409703             Usada
7  2024  163230324  Créditos (todos)
5  2024    2238553             Nueva
5  2024    1593760             Usada
0  2025  205077968  Créditos (todos)
6  2025    2301376             Nueva
6  2025    1783425             Usada
1  2026  217799437  Créditos (todos)
7  2026    2418399             Nueva
7  2026    1928446             Usada


In [97]:
import os
import plotly.graph_objects as go
import plotly.io as pio

# Crear una función para formatear los valores grandes en palabras en español
def formatear_numero(valor):
    if valor >= 1_000_000_000:
        return f'{valor / 1_000_000_000:.0f} mil millones'  # Mil millones
    elif valor >= 1_000_000:
        return f'{valor / 1_000_000:.0f} millones'  # Millones
    elif valor >= 1_000:
        return f'{valor / 1_000:.0f} mil'  # Mil
    else:
        return str(valor)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers+text',  # Mostrar líneas, puntos y texto
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center"  # Posicionar el texto encima de los puntos
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10)
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [1_000, 1_000_000, 1_000_000_000,5_000_000_000,10_000_000_000,15_000_000_000,20_000_000_000,25_000_000_000, 30_000_000_000]  # Valores hasta 30 mil millones
ticktext = [formatear_numero(val) for val in tickvals]  # Formatea cada valor en palabras

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()


# DIRECTOS

In [6]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'año': [2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026],
    'modalidad':['Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada'],    
    'montos': ['$1,430,769,432,.43','$1,533,664,464.12','$1,906,657,362.09','$1,850,699,331.67','$1,850,699,331.67','$1,850,699,331.67','$1,850,699,331.67','$1,850,699,331.67','$999,430,693.08','$1,003,135,857.15','$1,048,898,294.85','$1,014,220,724.54','$1,014,220,724.54','$1,014,220,724.54','$1,014,220,724.54','$1,014,220,724.54','$317,261,438.05','$357,257,543.98','$554,243,624.33','$628,801,488.21','$628,801,488.21','$628,801,488.21','$628,801,488.21','$628,801,488.21'],    
}
# Convertir a DataFrame
conteo_pachuca = pd.DataFrame(data)

In [26]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'año': [2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026],
    'modalidad':['Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada'],    
    'montos': [1430769432.43,1533664464.12,1906657362.09,1850699331.67,1850699331.67,1850699331.67,1850699331.67,1850699331.67,999430693.08,1003135857.15,1048898294.85,1014220724.54,1014220724.54,1014220724.54,1014220724.54,1014220724.54,317261438.05,357257543.98,554243624.33,628801488.21,628801488.21,628801488.21,628801488.21,628801488.21],    
}
# Convertir a DataFrame
conteo_pachuca = pd.DataFrame(data)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers',  # Mostrar líneas y puntos
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados        
        textposition="top center",  # Posicionar el texto encima de los puntos
        hovertemplate='%{text}<extra></extra>',  # Configurar el hover para mostrar solo el monto formateado
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10),
    legend=dict(
        orientation="h",
        y=1.2,  # Ajustar la posición de la leyenda para que no se sobreponga
        x=0.5,
        xanchor='center'
    )
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [500_000_000, 1_000_000_000, 1_500_000_000, 2_000_000_000]  # Valores especificados
ticktext = [f'${val:,.0f}' for val in tickvals]  # Formatear los valores para mostrarlos en el eje Y

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_total', carpeta='assets/graficas')
fig.show()

PROMEDIO

In [25]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'año': [2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026,2019,2020,2021,2022,2023,2024,2025,2026],
    'modalidad':['Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Créditos (todos)','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Nueva','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada','Vivienda Usada'],    
    'montos': [774645.06,980603.88,1087653.94,1469975.64,1469975.64,1469975.64,1469975.64,1469975.64,1024006.86,1227828.47,1452767.72,1635839.88,1635839.88,1635839.88,1635839.88,1635839.88,906461.25,1167508.31,1466252.97,2061644.22,2061644.22,2061644.22,2061644.22,2061644.22],    
}
# Convertir a DataFrame
conteo_pachuca = pd.DataFrame(data)

# Crear la figura
fig = go.Figure()

# Graficar cada modalidad
for modalidad in conteo_pachuca['modalidad'].unique():
    df_modalidad = conteo_pachuca[conteo_pachuca['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['año'], 
        y=df_modalidad['montos'], 
        mode='lines+markers',  # Mostrar líneas y puntos
        name=modalidad, 
        text=df_modalidad['montos'].apply(lambda x: f'${x:,.0f}'),  # Añadir el texto de los valores formateados
        textposition="top center",  # Posicionar el texto encima de los puntos
        hovertemplate='%{text}<extra></extra>',  # Configurar el hover para mostrar solo el monto formateado
    ))

# Configurar el diseño
fig.update_layout(
    xaxis_title="Año",
    yaxis_title="Monto total anual",
    template="plotly_white",
    margin=dict(l=10, r=10, t=10, b=10),
    legend=dict(
        orientation="h",
        y=1.2,  # Ajustar la posición de la leyenda para que no se sobreponga
        x=0.5,
        xanchor='center'
    )
)

# Configurar el eje Y utilizando el formateo personalizado
tickvals = [500_000, 1_000_000, 1_500_000, 2_000_000,2_500_000]  # Valores especificados
ticktext = [f'${val:,.0f}' for val in tickvals]  # Formatear los valores para mostrarlos en el eje Y

fig.update_yaxes(
    tickvals=tickvals, 
    ticktext=ticktext
)

# Función para exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar
guardar_grafico_como_html(fig, 'g_scatt_forecast_credit_promedio', carpeta='assets/graficas')
fig.show()